In [5]:
!pip install opencv-python

In [47]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import pandas as pd

from tqdm.auto import tqdm

import numpy as np
import scipy.linalg as sla
from PIL import Image
from scipy import misc
import scipy.sparse as sps
import scipy.sparse.linalg as spla
import timeit
%matplotlib inline

In [77]:
def prepare_photo_aug2(df, dir):
    """INPUT
    df -> pandas DataFrame: empty df
    dir -> str: abs path to dir with photo
        
        OTPUT -> pandas:DataFrame: full df"""
        
    #path = '/content/drive/MyDrive/НИС'+f'/{dir}'
    path = f"data/{dir}"
    photos = os.listdir(path)
    print('Кол-во фотографий', len(photos))

    for i, photo in enumerate(tqdm(photos)):
        try:
            photo_pth = path + '/' + photo
            img = cv2.imread(photo_pth, cv2.IMREAD_GRAYSCALE) # это матрица из интенсивностей серого цвета
            width = 224
            height = 224
            dim = (width, height)
            # resize image
            #resized = cv2.resize(img, dim, interpolation = cv2.INTER_LANCZOS4)
            data0 = np.reshape(img, 50176)
            data0 = pd.Series(data0)
            df = df.append(data0, ignore_index=True)
            
            '''
            center = (width/2, height/2)
            
            # добавим 90
            m = cv2.getRotationMatrix2D(center, 90, 1.0)
            rotated = cv2.warpAffine(img, m, dim)
            
            data = np.reshape(rotated, 50176)
            data = pd.Series(data)
            df = df.append(data, ignore_index=True)
            
            # добавим 180
            m2 = cv2.getRotationMatrix2D(center, 180, 1.0)
            rotated2 = cv2.warpAffine(img, m2, dim)
            data2 = np.reshape(rotated2, 50176)
            data2 = pd.Series(data2)
            df = df.append(data2, ignore_index=True)
            
            # добавим 270
            m3 = cv2.getRotationMatrix2D(center, 270, 1.0)
            rotated3 = cv2.warpAffine(img, m3, dim)      
            data3 = np.reshape(rotated3, 50176)
            data3 = pd.Series(data3)
            df = df.append(data3, ignore_index=True)
                                                    '''
        except:
             print(f'ALARM error with {i, photo}')
    return df

In [132]:
df_z = pd.DataFrame(columns=[i for i in range(50176)], dtype=float)

In [133]:
p1 = '1'  #predupr
p2 = '2'       #prior
p3 = '3'      #zapret
p4 = '4'   #predpis
p5 = '5' #os_predpis
p6 = '6'   #info
p7 = '7'          #service
p8 = '8'#tables

In [134]:
df1 = prepare_photo_aug2(df_z, p1)
df2 = prepare_photo_aug2(df_z, p2)
df3 = prepare_photo_aug2(df_z, p3)
df4 = prepare_photo_aug2(df_z, p4)
df5 = prepare_photo_aug2(df_z, p5)
df6 = prepare_photo_aug2(df_z, p6)
df7 = prepare_photo_aug2(df_z, p7)
df8 = prepare_photo_aug2(df_z, p8)

Кол-во фотографий 84


  0%|          | 0/84 [00:00<?, ?it/s]

Кол-во фотографий 35


  0%|          | 0/35 [00:00<?, ?it/s]

Кол-во фотографий 124


...

  0%|          | 0/81 [00:00<?, ?it/s]

Кол-во фотографий 28


  0%|          | 0/28 [00:00<?, ?it/s]

Кол-во фотографий 148


  0%|          | 0/148 [00:00<?, ?it/s]

In [135]:
df1['target'] = 'predupr'
df2['target'] = 'prior'
df3['target'] = 'zapret'
df4['target'] = 'predpis'
df5['target'] = 'ospredpis'
df6['target'] = 'info'
df7['target'] = 'service'
df8['target'] = 'tables'

In [136]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=0, ignore_index=True)

In [137]:
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_confusion_matrix

In [138]:
# Закодируем лейблы
LE = LabelEncoder()
df.target = LE.fit_transform(df.target)

# Разделим выборку
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.target, test_size=0.3, random_state=42)

In [148]:
import lightgbm
import catboost
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [144]:
catb = catboost.CatBoostClassifier(iterations=150, verbose=True, max_depth=10, learning_rate=0.1)

catb.fit(X_train, y_train)
y_pred_cat = catb.predict(X_test)

KeyboardInterrupt: 

In [160]:
train_dataset = lightgbm.Dataset(X_train, y_train)
test_dataset = lightgbm.Dataset(X_test, y_test)
#setting up the parameters
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='multiclass' #Multi-class target feature
params['metric']='multi_logloss' #metric for multi-class
params['max_depth']=10
params['num_class']=8 #no.of unique values in the target class not inclusive of the end value


lgbm = lightgbm.train(params, train_dataset, valid_sets=(test_dataset,))
# lgbm = lightgbm.train({"objective": "multiclass"},
#                     train_set=train_dataset, valid_sets=(test_dataset,),
#                     verbose_eval = -1)
y_pred_lgbm = lgbm.predict(X_test)
predictions = []

for x in y_pred_lgbm:
    predictions.append(np.argmax(x))

[1]	valid_0's multi_logloss: 1.89256
[2]	valid_0's multi_logloss: 1.83836
[3]	valid_0's multi_logloss: 1.78816
[4]	valid_0's multi_logloss: 1.74287
[5]	valid_0's multi_logloss: 1.70202
[6]	valid_0's multi_logloss: 1.66377
[7]	valid_0's multi_logloss: 1.62767
[8]	valid_0's multi_logloss: 1.59371
[9]	valid_0's multi_logloss: 1.56334
[10]	valid_0's multi_logloss: 1.53264
[11]	valid_0's multi_logloss: 1.50416
[12]	valid_0's multi_logloss: 1.47793
[13]	valid_0's multi_logloss: 1.45159
[14]	valid_0's multi_logloss: 1.42838
[15]	valid_0's multi_logloss: 1.40653
[16]	valid_0's multi_logloss: 1.3854
[17]	valid_0's multi_logloss: 1.36589
[18]	valid_0's multi_logloss: 1.34587
[19]	valid_0's multi_logloss: 1.32793
[20]	valid_0's multi_logloss: 1.31026
[21]	valid_0's multi_logloss: 1.29372
[22]	valid_0's multi_logloss: 1.27752
[23]	valid_0's multi_logloss: 1.26199
[24]	valid_0's multi_logloss: 1.24578
[25]	valid_0's multi_logloss: 1.23104
[26]	valid_0's multi_logloss: 1.21633
[27]	valid_0's multi_l

In [167]:
accuracy_score(y_test, predictions)

0.7971014492753623

In [152]:
xgb_clf = xgb.XGBClassifier(learning_rate=0.1, n_estimators=15, max_depth=4)
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)

C:\Users\Kerosene\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:45:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [153]:
accuracy_score(y_test, y_pred_xgb)

0.7536231884057971